In [12]:
import time
from tqdm import tqdm
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os

Determine the scaling factor for the scanning range

In [11]:
high_finesse_wavemeter_remote.start_acquisition()
high_finesse_wavemeter_remote._wavelength_buffer = []
wave_buff = high_finesse_wavemeter_remote.get_wavelength_buffer()
factor_sigma = float((np.max(np.array(wave_buff)) - np.min(np.array(wave_buff)) )/ 0.02)
factor_sigma

0

In [2]:
factor_sigma = 1.04

In [125]:
def go_to_poi(poi_cur, ref_poi="ref"):
    poi_manager_logic.go_to_poi(ref_poi)
    time.sleep(2)
    poi_manager_logic.go_to_poi(poi_cur)
    time.sleep(2)
    poi_manager_logic.go_to_poi(poi_cur)
    time.sleep(2)
    for i in range(2):
        scanning_optimize_logic.start_optimize()
        while scanning_optimize_logic.module_state()=='locked':
            time.sleep(1)
    time.sleep(1)

def optimize_ple():
    for i in range(2):
        ple_gui.sigToggleOptimize.emit(True)
        while ple_optimize_logic.module_state()=='locked':
            time.sleep(1)

def optimize_all():
    #assume the resonant is on
    optimize_ple()
    time.sleep(1)
    for i in range(2):
        scanning_optimize_logic.start_optimize()
        while scanning_optimize_logic.module_state()=='locked':
            time.sleep(1)
    time.sleep(1)
def set_resonant_power(power):
    ple_gui._mw.Controller_widget.power_SpinBox.setValue(power)
    time.sleep(0.1)
    ple_gui._mw.Controller_widget.power_SpinBox.editingFinished.emit()
    time.sleep(2)

def do_ple_scan(lines = 1, in_range = None, frequency=None, resolution=None):
    """
    fine_scan_range = (
            ple_gui.fit_result[1].best_values['center'] - ple_gui.fit_result[1].best_values['sigma'] * 3,
            ple_gui.fit_result[1].best_values['center'] + ple_gui.fit_result[1].best_values['sigma']  * 3
        )
    """
    if in_range is None:
        ple_gui._mw.actionFull_range.triggered.emit()
    else:
        ple_gui.sigScanSettingsChanged.emit(
             {
             'range': {ple_gui.scan_axis: in_range}
              }
        )
    ple_gui._mw.number_of_repeats_SpinBox.setValue(lines)
    ple_gui._mw.number_of_repeats_SpinBox.editingFinished.emit()
    time.sleep(0.5)
    ple_gui._mw.actionToggle_scan.setChecked(True)
    ple_gui.toggle_scan()
    while laser_scanner_logic.module_state()=='locked':
            time.sleep(1)
    time.sleep(1)
    ple_gui._fit_dockwidget.fit_widget.sigDoFit.emit("Lorentzian")
    time.sleep(1)
    # ple_gui._accumulated_data.mean(axis=0)
    # ple_gui.fit_result[1].params["center"].value
    return ple_gui.fit_result[1].params

def take_spectrum():
    spectrometer.acquire_spectrum()
    while spectrometerlogic._acquisition_running:
            time.sleep(1)
    time.sleep(1)

def save_spectrum(name_tag, folder_path=None):
    if folder_name:
        spectrometer._save_folderpath = folder_path
    spectrometer.save_widget.saveTagLineEdit.setText(name_tag)
    # hit save
    spectrometer.sigSaveSpectrum.emit()


def go_to_ple_target(target):
    #target = ple_gui.fit_result[1].params["center"].value
    ple_gui._mw.ple_widget.target_point.setValue(0)
    time.sleep(2)
    ple_gui._mw.ple_widget.target_point.setValue(target)
    ple_gui._mw.ple_widget.target_point.sigPositionChangeFinished.emit(target)
    time.sleep(2)

def one_pulse_repump(color='blue'):
    if color == "blue":
        cobolt.set_laser_modulated_power(power = 20)
        cobolt.enable_modulated()
        time.sleep(0.2)
        cobolt.disable_modulated()
    else:
        ibeam_smart.enable()
        time.sleep(0.2)
        ibeam_smart.disable()
def save_ple(tag, poi_name=None, folder_name = None):
    if folder_name:
        ple_gui._save_folderpath = folder_name
    ple_gui.save_path_widget.saveTagLineEdit.setText(
        f"{poi_name}_{tag}"
        )
    ple_gui._mw.actionSave.triggered.emit()

In [5]:
set_resonant_power(power = 290)

In [7]:
#set max resonant power
set_resonant_power(power = 300)
# turn on the CW repump
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 10)
#perform one ple scan to determine if there is a PLE
res = do_ple_scan(lines = 1, in_range = None)



go_to_ple_target(res["center"].value)





res = do_ple_scan(lines = 1)

fine_scan_range = ( 
    res["center"].value - 5 * res["sigma"].value ,
    res["center"].value + 5 * res["sigma"].value
                   )
set_resonant_power(power = 270)

res = do_ple_scan(lines = 1, in_range=fine_scan_range)

fine_scan_range = ( 
    res["center"].value - 5 * res["sigma"].value ,
    res["center"].value + 5 * res["sigma"].value
                   )

res = do_ple_scan(lines = 5, in_range=fine_scan_range)

In [15]:
if res["center"].stderr < 1e3 and res["sigma"].stderr <= res["sigma"].value: 
    print("Have ple")
else:
        #next poi
    print("No ple")

Have ple


In [17]:
one_pulse_repump(color="blue")

In [ ]:
cobolt.disable_modulated()

In [16]:
cobolt.disable_modulated()

## Do the experiment on one defect

In [58]:
import pandas as pd
import json

In [29]:
results_data = pd.DataFrame(columns=["poi_name",  "wavelength", "min_fwhm", "full_scan", "saturations", "xyz"])

In [30]:
results_poi = pd.DataFrame(columns=["poi_name",  "wavelength", "min_fwhm", "saturations", "xyz"])

In [51]:
high_finesse_wavemeter_remote.get_current_wavelength()

484.13082141008164

In [70]:
powers = [300, 270, 240, 220 , 200, 170, 150, 140, 130, 120, 110, 100, 80]
folder = r"C:\Users\yy3\Documents\data\SnV\cooldown_06_03_2023_95NA\197\2R\poiA"
os.mkdir(folder:=os.path.join(folder, r"automated_measurements"))

for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    os.mkdir(folder_defect := os.path.join(folder, poi_name))
    
    # go to a defect
    results_poi["poi_name"] = poi_name
    results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])
    ibeam_smart.enable()
    go_to_poi(poi_name)

    #set max resonant power
    set_resonant_power(power = 300)
    # turn on the CW repump
    ibeam_smart.disable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 10)
    #perform one ple scan to determine if there is a PLE
    res = do_ple_scan(lines = 1, in_range = None)
    if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value: 
        continue
    #since there is a PLE -- optimize_all
    go_to_ple_target(res["center"].value)
    #get the center wavelength:
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
    results_poi["wavelength, THz"] = float(wavelength)
    time.sleep(0.5)

    optimize_all()

    #do a san again to determine a finer range to scan
    res = do_ple_scan(lines = 5)
    #it makes sense to save here
    save_ple(tag = "full_range",
             poi_name=poi_name, folder_name=folder_defect)

    fine_scan_range = ( 
        res["center"].value - 5 * res["sigma"].value ,
        res["center"].value + 5 * res["sigma"].value
                    )
    set_resonant_power(power = 270)

    res = do_ple_scan(lines = 5, in_range=fine_scan_range)

    save_ple(tag = "5sigma_range",
             poi_name=poi_name, folder_name=folder_defect)

    fine_scan_range = ( 
        res["center"].value - 4 * res["sigma"].value ,
        res["center"].value + 4 * res["sigma"].value
                    )
    
    res = do_ple_scan(lines = 1, in_range=fine_scan_range)
    # here we have a somewhat fine range for the scan. 
    #Let's perform a saturation scan with a blue cw repump ON
    os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))

    idx_no_ple = None
    fwhms = []
    fwhms_errors = []
    for idx, power in enumerate(powers):
        go_to_ple_target(res["center"].value)
        set_resonant_power(power = power)
        optimize_ple()
        res = do_ple_scan(lines = 10, in_range=fine_scan_range)
        #save_results
        if res["sigma"].value is not None:
            fwhms.append(2 * res["sigma"].value)
        if res["sigma"].stderr is not None:
            fwhms_errors.append(2 * res["sigma"].stderr)
        save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=saturation_folder)
        #save_plots
        
        if res["center"].stderr > 1e3 or res["sigma"].stderr * 3 > res["sigma"].value: 
            #ple is gone.
            idx_no_ple = idx
            break
        results_poi.update({"saturation": 
                            {f"{power}":
                             {"data": }
                             [123,214,2354]}})
    results_poi["saturation_fwhm"] = float(fwhms)
    results_poi["saturation_fwhm_srderr"] = float(fwhms_errors)

    if idx_no_ple is None:
        continue
    # photostability study

    set_resonant_power(power = powers[idx_no_ple - 2]) #take the last visible ple power
    #reionize with blue
    os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))

  
    for i in range(3):
        #walk off resonance:
        go_to_ple_target(0)
        time.sleep(0.5)
        cobolt.enable_modulated()
        cobolt.set_laser_modulated_power(power = 20)
        time.sleep(2)
        cobolt.disable_modulated()
        ibeam_smart.disable()
        res = do_ple_scan(lines = 50, in_range=fine_scan_range)
        #cobolt.set_laser_modulated_power(power = 10)
        save_ple(tag = "photostability", folder_name=photostability) 

    #reionize with green
    os.mkdir(photostability := os.path.join(folder_defect, "photostability_520"))
    for i in range(3):
        #walk off resonance:
        go_to_ple_target(0)
        time.sleep(0.5)
        ibeam_smart.enable()
        time.sleep(2)
        cobolt.disable_modulated()
        ibeam_smart.disable()
        
        res = do_ple_scan(lines = 20, in_range=fine_scan_range)
        #cobolt.set_laser_modulated_power(power = 10)
        save_ple(tag = "photostability", folder_name=photostability) 
    
    with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
        file.write(json.dumps(results_poi)) # use `json.loads` to do the reverse
    #with open(os.path.join(folder_defect, f'results_{poi_name}.txt')) as file:
    #results = json.load(file) # use `json.loads` to do the reverse

In [94]:
#results_poi["xyz"] = tuple(float(xx) for xx in results_poi["xyz"] )

True

False

In [96]:
results_poi["xyz"]

(1.1754241822507253e-05, 1.8739329866488726e-05, 6.222e-07)

In [108]:
results_poi

{'poi_name': 'def51', 'xyz': (1.1754241822507253e-05, 1.8739329866488726e-05, 6.222e-07)}

In [112]:
results_poi.update({"saturation": {"100a.u.":[123,214,2354]}})

In [114]:
results_poi["saturation"]["100a.u."]

[123, 214, 2354]

In [105]:
with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
    file.write(json.dumps(results_poi)) # use `json.loads` to do the reverse

In [106]:
with open(os.path.join(folder_defect, f'results_{poi_name}.txt')) as file:
    results = json.load(file) # use `json.loads` to do the reverse

In [79]:
ibeam_smart.disable()

In [72]:
cobolt.disable_modulated()

In [73]:
cobolt.enable_modulated()

In [87]:
ibeam_smart.enable()